In [5]:
from decaptcher import Decaptcher
from string import Template

import requests
import re

sid = 'gdq6vb9b1q3ufg85hvudn3koh7'
headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'Origin': 'https://tempr.email',
    'Content-Type': 'application/x-www-form-urlencoded',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://tempr.email/password-action=remove.htm',
    'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8'
}

cookies = {
    'sid': sid,
    '_ga': 'GA1.2.85922033.1606315288',
    'LocalPart': '1',
    'DomainId': '7452',
    '_gid': 'GA1.2.1179053531.1608475621',
    '_gat': '1'
}

class DecaptcherTester:
    set_password_url = 'https://tempr.email/password-action=set.htm'
    remove_password_url = 'https://tempr.email/password-action=remove.htm'
    
    set_password_payload = {
        'SetPassword': '123456',
        'secureCaptcha': '',
        'SetPasswordButton': 'Passwort setzen'
    }
    
    remove_password_payload = {
        'RemovePasswordButton': 'Passwort+entfernen'
    }
    
    def __init__(self, sid, headers, cookies, decaptcher):
        self.sid = sid
        self.headers = headers
        self.cookies = cookies
        self.decaptcher = decaptcher
        self.captcha_url = f'https://tempr.email/application/api/secureCaptcha.php?sid={sid}&small=1'
        self.reset_counters()
    
    def reset_counters(self):
        self.correct_answers_count = 0
        self.incorrect_answers_count = 0
        self.check_count = 0
    
    def set_password(self):
        guess = self.decaptcher.solve_captcha(self.captcha_url).strip().lower()
        print(guess)
        if len(guess) != 3:
            return False
        self.set_password_payload['secureCaptcha'] = guess
        set_password_response = requests.post(self.set_password_url, headers = self.headers, cookies = self.cookies, data = self.set_password_payload)
        if 'Passwort wurde erfolgreich gesetzt' in set_password_response.text:
            return True
        elif 'Falsche Eingabe des CAPTCHA' in set_password_response.text:
            return False
        else:
            print('?')
            # print(set_password_response.text)
            return False

    def remove_password(self):
        requests.post(self.remove_password_url, headers = self.headers, cookies = self.cookies, data = self.remove_password_payload)
    
    def check(self):
        self.check_count += 1
        if self.set_password():
            self.correct_answers_count += 1
        else:
            self.incorrect_answers_count += 1   
        self.remove_password()
    
    def check_n_times(self, n):
        self.reset_counters()
        for i in range(n):
            self.check()
            accuracy = round(self.correct_answers_count / self.check_count, 2) * 100
            print(f'[{self.correct_answers_count}/{self.check_count}] Guessing accuracy: {accuracy}%')

decaptcher = Decaptcher(sid, headers, cookies)
decaptcher_tester = DecaptcherTester(sid, headers, cookies, decaptcher)

decaptcher_tester.check_n_times(3)

asb
[0/1] Guessing accuracy: 0.0%
tcl
[0/2] Guessing accuracy: 0.0%
gdn
[1/3] Guessing accuracy: 33.0%
